<h2><a href="https://leetcode.com/problems/check-if-digits-are-equal-in-string-after-operations-i">3461. Check If Digits Are Equal in String After Operations I</a></h2><h3>Easy</h3><hr><p>You are given a string <code>s</code> consisting of digits. Perform the following operation repeatedly until the string has <strong>exactly</strong> two digits:</p>

<ul>
	<li>For each pair of consecutive digits in <code>s</code>, starting from the first digit, calculate a new digit as the sum of the two digits <strong>modulo</strong> 10.</li>
	<li>Replace <code>s</code> with the sequence of newly calculated digits, <em>maintaining the order</em> in which they are computed.</li>
</ul>

<p>Return <code>true</code> if the final two digits in <code>s</code> are the <strong>same</strong>; otherwise, return <code>false</code>.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<div class="example-block">
<p><strong>Input:</strong> <span class="example-io">s = &quot;3902&quot;</span></p>

<p><strong>Output:</strong> <span class="example-io">true</span></p>

<p><strong>Explanation:</strong></p>

<ul>
	<li>Initially, <code>s = &quot;3902&quot;</code></li>
	<li>First operation:
	<ul>
		<li><code>(s[0] + s[1]) % 10 = (3 + 9) % 10 = 2</code></li>
		<li><code>(s[1] + s[2]) % 10 = (9 + 0) % 10 = 9</code></li>
		<li><code>(s[2] + s[3]) % 10 = (0 + 2) % 10 = 2</code></li>
		<li><code>s</code> becomes <code>&quot;292&quot;</code></li>
	</ul>
	</li>
	<li>Second operation:
	<ul>
		<li><code>(s[0] + s[1]) % 10 = (2 + 9) % 10 = 1</code></li>
		<li><code>(s[1] + s[2]) % 10 = (9 + 2) % 10 = 1</code></li>
		<li><code>s</code> becomes <code>&quot;11&quot;</code></li>
	</ul>
	</li>
	<li>Since the digits in <code>&quot;11&quot;</code> are the same, the output is <code>true</code>.</li>
</ul>
</div>

<p><strong class="example">Example 2:</strong></p>

<div class="example-block">
<p><strong>Input:</strong> <span class="example-io">s = &quot;34789&quot;</span></p>

<p><strong>Output:</strong> <span class="example-io">false</span></p>

<p><strong>Explanation:</strong></p>

<ul>
	<li>Initially, <code>s = &quot;34789&quot;</code>.</li>
	<li>After the first operation, <code>s = &quot;7157&quot;</code>.</li>
	<li>After the second operation, <code>s = &quot;862&quot;</code>.</li>
	<li>After the third operation, <code>s = &quot;48&quot;</code>.</li>
	<li>Since <code>&#39;4&#39; != &#39;8&#39;</code>, the output is <code>false</code>.</li>
</ul>
</div>

<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>3 &lt;= s.length &lt;= 100</code></li>
	<li><code>s</code> consists of only digits.</li>
</ul>


## 1. Intuition 💡

The "obvious" solution is to simulate the process, but that's slow ($O(n^2)$). The problem asks us to repeat an operation `(a+b)%10` over and over. This repeated summing hints at a mathematical pattern.

Let's trace how much each *original* digit contributes to the *final* digits.

**Example: `s = "abcde"` ($n=5$)**

* **Initial:** `a, b, c, d, e`
* **Step 1:** `(a+b), (b+c), (c+d), (d+e)`
* **Step 2:** `(a+b) + (b+c), (b+c) + (c+d), (c+d) + (d+e)`
    * This simplifies to: `(a + 2b + c), (b + 2c + d), (c + 2d + e)`
* **Step 3 (Final):** `(a+2b+c) + (b+2c+d), (b+2c+d) + (c+2d+e)`
    * **Final Digit 1:** `a + 3b + 3c + d`
    * **Final Digit 2:** `b + 3c + 3d + e`

(Note: All math is modulo 10, but we'll ignore that for a second to see the pattern.)

Look at the coefficients:
* For Final Digit 1: **`[1, 3, 3, 1]`**
* For Final Digit 2: **`[1, 3, 3, 1]`**

The coefficients **`1, 3, 3, 1`** are the **3rd row of Pascal's triangle**. 

Why the 3rd row? Because for $n=5$, we perform $k = n - 2 = 3$ steps of the operation.

The core intuition is this: **We don't need to *simulate* the process. We can jump directly to the end!** The problem is just a weighted sum.

For any string of length $n$, we will perform $k = n - 2$ operations. The weights for the sum are the coefficients of the $k$-th row of Pascal's triangle.

* `final_digit_1` = $(C(k, 0) \cdot s[0] + C(k, 1) \cdot s[1] + \dots + C(k, k) \cdot s[k]) \pmod{10}$
* `final_digit_2` = $(C(k, 0) \cdot s[1] + C(k, 1) \cdot s[2] + \dots + C(k, k) \cdot s[k+1]) \pmod{10}$

(Where $C(k, i)$ is the binomial coefficient "k choose i", i.e., the $i$-th number in the $k$-th row).

Our goal is to calculate these two sums and check if they are equal.

---

## 2. Approach 🗺️

1.  Get the length of the string, $n$.
2.  Calculate the number of operations, $k = n - 2$. This is the row of Pascal's triangle we care about.
3.  Convert the input string $s$ into a list of integers called `digits`.
4.  Generate the $k$-th row of Pascal's triangle. This row will have $k+1$ (or $n-1$) numbers. We can do this efficiently in $O(n)$ time using the formula:
    * $C(k, 0) = 1$
    * $C(k, i) = C(k, i-1) \cdot (k - i + 1) / i$
5.  Store these $n-1$ coefficients in a list called `coeffs`.
6.  Initialize two variables, `d1 = 0` and `d2 = 0`, to store the sums for our two final digits.
7.  Iterate $i$ from $0$ to $n-2$ (which is $k+1$ times, the length of our `coeffs` list).
    * Get the current coefficient: `c = coeffs[i] % 10`. We only need its last digit for the modulo 10 sum.
    * Calculate `d1`: `d1 = (d1 + c * digits[i]) % 10`
    * Calculate `d2`: `d2 = (d2 + c * digits[i+1]) % 10`
8.  After the loop finishes, `d1` and `d2` hold the values of the two final digits.
9.  Return `d1 == d2`.

---

## 4. Time Complexity ⏱️

**$O(n)$**

* `digits = [int(d) for d in s]`: This is a single pass over the string of length $n$. **$O(n)$**.
* `coeffs = [1] * (n - 1)`: Initializing the list takes **$O(n)$** time.
* Generating `coeffs` loop: `for i in range(1, k + 1)`: This loop runs $k$ times. Since $k = n - 2$, this loop runs $O(n)$ times. Each calculation inside is $O(1)$. Total: **$O(n)$**.
* Calculating `d1` and `d2` loop: `for i in range(n - 1)`: This loop runs $n-1$ times. Each calculation inside is $O(1)$. Total: **$O(n)$**.

The overall time complexity is $O(n) + O(n) + O(n) + O(n)$, which simplifies to **$O(n)$**.

---

## 5. Space Complexity 💾

**$O(n)$**

* `digits` list: We create a list of integers of length $n$. This takes **$O(n)$** space.
* `coeffs` list: We create a list of coefficients of length $n-1$. This takes **$O(n)$** space.
* All other variables ($n$, $k$, $i$, `d1`, `d2`, `coeff_mod_10`) are single numbers, taking $O(1)$ constant space.

The overall space complexity is dominated by the two lists we create, making it $O(n) + O(n) = $**$O(n)$**.

---

## Code

```python
class Solution:
    def hasSameDigits(self, s: str) -> bool:
        
        n = len(s)
        # k is the number of operations, and the row of Pascal's triangle
        k = n - 2
        
        # 1. Convert string to a list of integers
        digits = [int(d) for d in s]
        
        # 2. Generate the k-th row of Pascal's triangle
        # This row will have k+1 = n-1 elements.
        # We initialize it with 1s, but we'll overwrite.
        coeffs = [1] * (n - 1)
        
        # We calculate C(k, i) using C(k, i-1)
        # This loop runs k times (O(n))
        for i in range(1, k + 1):
            # This is the formula: C(k,i) = C(k,i-1) * (k - i + 1) // i
            # It calculates the next coefficient from the previous one
            coeffs[i] = (coeffs[i-1] * (k - i + 1) // i)
            
        # 3. Calculate the two final digits using the coefficients
        d1 = 0
        d2 = 0
        
        # This loop runs n-1 times (O(n))
        for i in range(n - 1): # range(n-1) is the same as range(k+1)
            
            # We only need the coefficient mod 10 for the sum
            coeff_mod_10 = coeffs[i] % 10
            
            # Sum for the first final digit
            # (a + 3b + 3c + d) % 10
            d1 = (d1 + coeff_mod_10 * digits[i]) % 10
            
            # Sum for the second final digit, using the shifted digits
            # (b + 3c + 3d + e) % 10
            d2 = (d2 + coeff_mod_10 * digits[i+1]) % 10
            
        # 4. Return the final comparison
        return d1 == d2

In [4]:
#Pascal's Triangle
class Solution:
    def hasSameDigits(self, s: str) -> bool:
        n = len(s)
        k = n - 2
        
        digits = [ord(ch) - 48 for ch in s]

        coeffs = [1] * (n - 1)
        
        for i in range(1, k + 1):
            coeffs[i] = (coeffs[i-1] * (k - i + 1) // i)
            
        d1 = 0
        d2 = 0
        
        for i in range(n - 1): 
            coeff_mod_10 = coeffs[i] % 10
            d1 = (d1 + coeff_mod_10 * digits[i]) % 10
            d2 = (d2 + coeff_mod_10 * digits[i+1]) % 10
        
        return d1 == d2

result = Solution().hasSameDigits("3902")
print(result)

True


## Explanation of the Simulation Approach

### Intuition and Approach 💡
The problem asks us to repeatedly perform the operation `(a + b) % 10` on consecutive digits of a string until only two digits remain. The simulation approach directly mimics this process:
1. Convert the string into a list of integers.
2. Repeatedly calculate the new digits using `(a + b) % 10` for all consecutive pairs.
3. Replace the original list with the newly calculated digits.
4. Stop when the list has exactly two digits.
5. Compare the two digits and return whether they are equal.

This approach is straightforward and easy to implement but has a higher time complexity due to repeated iterations over the list.

---

### Code Explanation in Detail 📝

#### Step-by-Step Breakdown:
1. **Convert String to Integers:**
   ```python
   s_int = [ord(ch) - 48 for ch in s]
   ```
   This converts each character in the string `s` into its corresponding integer value.

2. **Iterative Simulation:**
   ```python
   while len(s_int) != 2:
       l = len(s_int)
       ans = []
       for i in range(l-1):
           temp = (s_int[i] + s_int[i+1]) % 10
           ans.append(temp)
       s_int = ans
   ```
   - While the list has more than two digits, calculate the new digits for all consecutive pairs.
   - Store the results in a temporary list `ans`.
   - Replace `s_int` with `ans` for the next iteration.

3. **Final Comparison:**
   ```python
   if s_int[0] == s_int[1]:
       return True
   else:
       return False
   ```
   - Once the list has exactly two digits, compare them.
   - Return `True` if they are equal, otherwise `False`.

---

### Dry Run 🧪

#### Example: `s = "3902"`
1. **Initial Conversion:**
   ```python
   s_int = [3, 9, 0, 2]
   ```

2. **First Iteration:**
   - Calculate new digits:
     ```python
     (3 + 9) % 10 = 2
     (9 + 0) % 10 = 9
     (0 + 2) % 10 = 2
     ```
   - Update `s_int`:
     ```python
     s_int = [2, 9, 2]
     ```

3. **Second Iteration:**
   - Calculate new digits:
     ```python
     (2 + 9) % 10 = 1
     (9 + 2) % 10 = 1
     ```
   - Update `s_int`:
     ```python
     s_int = [1, 1]
     ```

4. **Final Comparison:**
   ```python
   s_int[0] == s_int[1]  # True
   ```
   - Return `True`.

---

### Edge Cases ⚠️

1. **Zero Input:**
   - If `s` is empty, the function cannot proceed. However, the constraints ensure `s.length >= 3`.

2. **Negative Numbers:**
   - Not applicable since `s` consists only of digits.

3. **Smallest Input:**
   - For `s = "123"`, the function performs one iteration and directly compares the two resulting digits.

4. **Largest Input:**
   - For `s` with 100 digits, the function performs multiple iterations, reducing the list size by one in each iteration.

---

### Time Complexity ⏱️

#### Exact Analysis:
1. **Initial Conversion:**
   ```python
   s_int = [ord(ch) - 48 for ch in s]
   ```
   - Runs in $O(n)$, where $n$ is the length of `s`.

2. **Simulation Loop:**
   ```python
   while len(s_int) != 2:
   ```
   - In each iteration, the list size decreases by 1.
   - The total number of iterations is $n - 2$.
   - Each iteration involves calculating $O(n)$ pairs.
   - Total complexity: $O(n^2)$.

3. **Final Comparison:**
   ```python
   s_int[0] == s_int[1]
   ```
   - Runs in $O(1)$.

**Overall Time Complexity:**
- **Best Case:** $O(n^2)$
- **Average Case:** $O(n^2)$
- **Worst Case:** $O(n^2)$

---

### Space Complexity 💾

#### Exact Analysis:
1. **Input Conversion:**
   - The list `s_int` requires $O(n)$ space.

2. **Temporary List:**
   - The list `ans` requires $O(n)$ space in each iteration.

**Overall Space Complexity:**
- **Best Case:** $O(n)$
- **Average Case:** $O(n)$
- **Worst Case:** $O(n)$

In [2]:
# The Simulation Approach
class Solution:
    def hasSameDigits(self, s: str) -> bool:
        s_int = [ord(ch) - 48 for ch in s]
        while len(s_int) != 2:
            l = len(s_int)
            ans = []
            for i in range(l-1):
                temp = (s_int[i] + s_int[i+1]) % 10
                ans.append(temp)
            s_int = ans

        if s_int[0] == s_int[1]:
            return True
        else:
            return False

result = Solution().hasSameDigits("3902")
print(result)

True
